In [1]:
import MeCab
import itertools
import pprint

In [2]:
class wordvec_lib:
    
    def _word_attrs(self, txt):
        m = MeCab.Tagger("mecab-ipadic-neologd")
        line = m.parse(txt).split('\n')[:-2]
        return [l.split('\t')[1].split(',') for l in line]

    def _filter_words(self, word_attrs):
        # filter noun and verb
        w = word_attrs
        w = [m for m in w if m[0] in ['名詞', '動詞']]
        return w

    def _attr_to_word_vec(self, attrs):
        count = len(attrs)
        words = [a[6] for a in attrs]
        return {g[0] : len(list(g[1]))/count for g in itertools.groupby(sorted(words))}

    def get_word_vec(self, txt):
        attrs = self._word_attrs(txt)
        attrs = self._filter_words(attrs)
        return self._attr_to_word_vec(attrs)
    
    def dot_product(self, v1, v2):
        return sum(v1[key]*v2.get(key, 0) for key in v1)

In [3]:
class qa:
    
    w = wordvec_lib()
    
    def set_standard_qa(self, qa_list):
        self.qa_list = qa_list
        self.vec_db = [self.w.get_word_vec(q[0]) for q in qa_list]
        pp = pprint.PrettyPrinter(indent=4)
        print('=== word count vector ===')
        pp.pprint(self.vec_db)

    def q(self, q_txt):
        self.w = wordvec_lib()
        vec_q = self.w.get_word_vec(q_txt)
        inner_prods = [self.w.dot_product(v, vec_q) for v in self.vec_db]
        max_idx = inner_prods.index(max(inner_prods))
        return self.qa_list[max_idx][1]

In [4]:
qa_txt = [('照明のつけ方は？',          '照明をつけるにはボタン１を押します'),
          ('エアコンのつけ方は？',      'エアコンをつけるにはボタン２を押します'),
          ('エアコンの温度の上げ方は？', 'エアコンの温度を上げるにはボタン４を押します'),
          ('テレビのつけ方は？',        'テレビをつけるにはボタン３を押します')]
qa_elec = qa()
qa_elec.set_standard_qa(qa_txt)

=== word count vector ===
[   {   'つけ': 0.3333333333333333,
        '方': 0.3333333333333333,
        '照明': 0.3333333333333333},
    {   'つけ': 0.3333333333333333,
        'エアコン': 0.3333333333333333,
        '方': 0.3333333333333333},
    {'エアコン': 0.25, '上げ': 0.25, '方': 0.25, '温度': 0.25},
    {   'つけ': 0.3333333333333333,
        'テレビ': 0.3333333333333333,
        '方': 0.3333333333333333}]


In [5]:
qa_elec.q('エアコンの温度設定はどうやるの？')

'エアコンの温度を上げるにはボタン４を押します'

In [6]:
qa_elec.q('エアコンをつけたいんだけど')

'エアコンをつけるにはボタン２を押します'

In [7]:
qa_elec.q('テレビをつけるには？')

'テレビをつけるにはボタン３を押します'

In [8]:
qa_elec.q('どうやって照明をつけるの？')

'照明をつけるにはボタン１を押します'